<a href="https://colab.research.google.com/github/anshupandey/Working_with_Large_Language_models/blob/main/WWL_C6_transformer_based_classification_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning: Transformer based Vectorizer

In [ ]:
!pip install sentence-transformers --quiet

In [14]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd

In [15]:
# load dataset
url = "https://raw.githubusercontent.com/anshupandey/Working_with_Large_Language_models/main/sentimentdata.csv"
df = pd.read_csv(url)
df.shape

(20, 2)

In [16]:
df.head()

,document,label
0,Pizza is great and I love pizza.,Positive
1,I hate burger and its bad to eat burger.,Negative
2,I hate dirty tables.,Negative
3,Burger is amazing and I love it more than anyt...,Positive
4,My boss is a monster and I hate him,Negative


In [17]:
x = df['document']
y = df['label']


## Exploring sentence vectorizer

In [18]:
# Load the pre-trained SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Example vectors
vec1 = model.encode("Noodles are good")
vec2 = model.encode("Noodles are amazing")
print(vec1.shape,vec2.shape)


# Calculate cosine similarity
similarity = cosine_similarity(vec1.reshape(1,-1), vec2.reshape(1,-1))
print(f"Cosine Similarity: {similarity[0][0]}")


(384,) (384,)
Cosine Similarity: 0.9152442216873169


In [25]:
# Example vectors
vec1 = model.encode("Noodles are good")
vec2 = model.encode("Noodles are very bad")
print(vec1.shape,vec2.shape)


# Calculate cosine similarity
similarity = cosine_similarity(vec1.reshape(1,-1), vec2.reshape(1,-1))
print(f"Cosine Similarity: {similarity[0][0]}")

(384,) (384,)
Cosine Similarity: 0.857244610786438


## Transfer Learning: Training a sentiment classification model

In [26]:
# Convert texts to embeddings
X = model.encode(x)
y = np.array(y)

In [27]:
# Create and train the logistic regression model
clf = LogisticRegression()
clf.fit(X, y)


LogisticRegression()

## Evaluation on sample data

In [8]:
ip = "Noodles are good"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Positive'], dtype=object)

In [9]:
ip = "Noodles are not good"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Negative'], dtype=object)

In [10]:
ip = "Noodles are bad"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Negative'], dtype=object)

In [11]:
ip = "Noodles are not good"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Negative'], dtype=object)

In [12]:
ip = "Noodles are great"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Positive'], dtype=object)

In [13]:
ip = "Noodles are not great"
ipvec = model.encode([ip])
clf.predict(ipvec)

array(['Negative'], dtype=object)